In [2]:
! pip install cinemagoer


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from imdb import Cinemagoer

In [9]:
data = Cinemagoer()

In [11]:
data